#### **Ingesta de la carpeta "movie_company"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.functions import lit,current_timestamp
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

##### Esquema

In [0]:
movie_company_schema = StructType([
    StructField("movie_id",IntegerType(),False),
    StructField("company_id",StringType(),True)
])

##### Leer ficheros CSV

In [0]:
df_movie_company = spark.read.format("csv").schema(movie_company_schema).load(f"{bronze_folder_path}/{var_file_date}/movie_company")

##### Agregar columnas

In [0]:
df_movie_company_final = add_ingestion_date(df_movie_company).withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir en Silver

In [0]:
merge_condition = "target.movie_id = source.movie_id AND target.company_id = source.company_id AND target.file_date = source.file_date"
merge_delta_lake(df_movie_company_final,"movie_silver","movies_companies",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT COUNT(1), file_date
FROM movie_silver.movies_companies
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success")